In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# 1. Charger la base de données
df = pd.read_csv('base_anomalies_6082.csv')

# 2. Sélectionner les colonnes pertinentes
features = ['Statut de salariés', 'Frontalier', 'Rub 6082', '6082Taux']
target = 'anomalie_csg'

# 3. Convertir les variables catégorielles en numériques (Statut de salariés, Frontalier)
label_encoder_statut = LabelEncoder()
label_encoder_frontalier = LabelEncoder()

df['Statut de salariés'] = label_encoder_statut.fit_transform(df['Statut de salariés'])
df['Frontalier'] = label_encoder_frontalier.fit_transform(df['Frontalier'])

# 4. Préparer les données pour le modèle
X = df[features]
y = df[target]

# 5. Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 6. Créer et entraîner le modèle RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 7. Faire des prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)

# 8. Évaluer la performance du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy * 100))

# Rapport de classification pour plus de détails
print("Rapport de classification :\n", classification_report(y_test, y_pred))


Précision du modèle sur les données de test : 100.00%
Rapport de classification :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     87607
           1       1.00      1.00      1.00      9528

    accuracy                           1.00     97135
   macro avg       1.00      1.00      1.00     97135
weighted avg       1.00      1.00      1.00     97135



In [19]:
import joblib

# Exporter le modèle Random Forest entraîné
joblib.dump(rf_classifier, 'random_forest_model_6082.pkl')


['random_forest_model_6082.pkl']

In [27]:
import pandas as pd
import joblib

# 1. Charger le modèle entraîné
rf_classifier_loaded = joblib.load('random_forest_model_6082.pkl')

# 2. Charger la nouvelle base de données (sans la colonne 'anomalie_csg')
new_df = pd.read_csv('new_updated_file 2.csv')

# 3. Sélectionner les mêmes colonnes que celles utilisées pour l'entraînement
features_new = new_df[['Statut de salariés', 'Frontalier', 'Rub 6082', '6082Taux']]

# 4. Encodage des variables catégorielles (Statut de salariés, Frontalier)
# Vous devez utiliser les mêmes encodeurs que ceux utilisés lors de l'entraînement
label_encoder_statut = LabelEncoder()
label_encoder_frontalier = LabelEncoder()

# Encodage des variables catégorielles dans la nouvelle base
features_new['Statut de salariés'] = label_encoder_statut.fit_transform(features_new['Statut de salariés'])
features_new['Frontalier'] = label_encoder_frontalier.fit_transform(features_new['Frontalier'])

# 5. Vérifier que le nombre de colonnes correspond bien à celles utilisées pour l'entraînement
# Si certaines colonnes manquent (cas rare), on les ajoute avec des valeurs par défaut
missing_cols = set(X.columns) - set(features_new.columns)  # 'X' ici représente les colonnes d'entraînement
for c in missing_cols:
    features_new[c] = 0  # Ajouter des colonnes manquantes avec une valeur par défaut

# 6. Réarranger les colonnes dans le même ordre que lors de l'entraînement
features_new = features_new[X.columns]

# 7. Utiliser le modèle pour prédire les anomalies sur la nouvelle base
predictions = rf_classifier_loaded.predict(features_new)

# 8. Ajouter la colonne 'anomalie_predite' pour indiquer les prédictions faites par le modèle
new_df['anomalie_predite'] = predictions

# 9. Sauvegarder la nouvelle base de données avec les prédictions
new_df.to_csv('nouvelle_base_avec_predictions.csv', index=False)

print("Prédictions ajoutées à la nouvelle base de données.")


Prédictions ajoutées à la nouvelle base de données.


C:\Users\ouiss\AppData\Local\Temp\ipykernel_7932\275050220.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_new['Statut de salariés'] = label_encoder_statut.fit_transform(features_new['Statut de salariés'])
C:\Users\ouiss\AppData\Local\Temp\ipykernel_7932\275050220.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_new['Frontalier'] = label_encoder_frontalier.fit_transform(features_new['Frontalier'])


In [15]:
import pandas as pd
import joblib

# 1. Charger la nouvelle base de données
df_nouveau = pd.read_csv('base_anomalies_csg.csv')

# 2. Sélectionner les mêmes colonnes de caractéristiques que pour l'entraînement
features = ['Statut de salariés', 'Frontalier', 'Rub 6082', '6082Taux']

# 3. Convertir les variables catégorielles en numériques (Statut de salariés, Frontalier)
# Utiliser les mêmes encodeurs que pour l'entraînement
df_nouveau['Statut de salariés'] = label_encoder_statut.transform(df_nouveau['Statut de salariés'])
df_nouveau['Frontalier'] = label_encoder_frontalier.transform(df_nouveau['Frontalier'])

# 4. Préparer les données pour la prédiction
X_nouveau = df_nouveau[features]

# 5. Charger le modèle Random Forest précédemment entraîné
rf_classifier = joblib.load('random_forest_model_6082.pkl')

# 6. Faire des prédictions sur la nouvelle base de données
predictions = rf_classifier.predict(X_nouveau)

# 7. Ajouter les prédictions à la base de données
df_nouveau['anomalie_predite'] = predictions

# 8. Sauvegarder la base de données avec les prédictions
df_nouveau.to_csv('nouvelle_base_avec_anomalies.csv', index=False)

print("Les prédictions d'anomalies ont été ajoutées et la base de données a été sauvegardée.")


Les prédictions d'anomalies ont été ajoutées et la base de données a été sauvegardée.


In [38]:
import pandas as pd
import numpy as np

# 1. Charger la base de données existante
df = pd.read_csv('new_updated_file 2.csv')

# 2. Définir le taux d'anomalies (par exemple 10 % des lignes)
taux_anomalie = 0.10  # 10 % d'anomalies

# 3. Sélectionner aléatoirement un certain pourcentage de lignes pour introduire des anomalies
n_lignes = df.shape[0]
n_anomalies = int(taux_anomalie * n_lignes)

# 4. Sélectionner des indices aléatoires distincts pour les anomalies
indices_anomalies_csg = np.random.choice(df.index, size=n_anomalies, replace=False)

# 5. Ajouter une colonne pour marquer les anomalies
df['anomalie_csg'] = 0  # Initialement, aucune anomalie

# 6. Fonction pour générer un taux aléatoire erroné, arrondi à 2 décimales
def generer_taux_errone(eliminer_valeur=None, valeur_max=10):
    taux_errone = round(np.random.uniform(0, valeur_max), 2)  # Choisissez une plage appropriée et arrondissez
    while eliminer_valeur is not None and taux_errone == eliminer_valeur:
        taux_errone = round(np.random.uniform(0, valeur_max), 2)  # Générer jusqu'à ce ne soit pas la valeur à éliminer
    return taux_errone

# 7. Fonction pour introduire des anomalies dans la cotisation CSG
def introduire_anomalie_csg(row):
    frontalier = row['Frontalier']
    statut = row['Statut de salariés']

    # Conditions d'anomalie
    if frontalier == 'Frontaliers' and statut == 'Stagiaire':
        # Taux correct est 0%, générer un taux erroné
        row['6082Taux'] = generer_taux_errone(eliminer_valeur=0)
        row['anomalie_csg'] = 1
    elif frontalier == 'Frontaliers' and statut != 'Stagiaire':
        # Taux correct est 0%, générer un taux erroné
        row['6082Taux'] = generer_taux_errone(eliminer_valeur=0)
        row['anomalie_csg'] = 1
    elif frontalier == 'Non' and statut == 'Stagiaire':
        # Taux correct est 0%, générer un taux erroné
        row['6082Taux'] = generer_taux_errone(eliminer_valeur=0)
        row['anomalie_csg'] = 1
    elif frontalier == 'Non' and statut != 'Stagiaire':
        # Taux correct est 6.8%, générer un taux erroné
        row['6082Taux'] = generer_taux_errone(eliminer_valeur=6.8)
        row['anomalie_csg'] = 1

    return row

# 8. Appliquer la fonction d'anomalie uniquement sur les lignes sélectionnées
df.loc[indices_anomalies_csg, :] = df.loc[indices_anomalies_csg, :].apply(introduire_anomalie_csg, axis=1)

# 9. Sauvegarder la nouvelle base de données avec anomalies
df.to_csv('base_anomalies_csg.csv', index=False)

# Optionnel : Afficher un résumé des anomalies introduites
#total_anomalies_csg = df['anomalie_csg'].sum()
#print(f"Total des anomalies introduites pour la cotisation CSG : {total_anomalies_csg} sur {n_lignes} lignes ({(total_anomalies_csg/n_lignes)*100:.2f}%)")


test

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. Charger la base de données
df = pd.read_csv('base_anomalies_6082.csv')

# 2. Sélectionner les colonnes pertinentes
features = ['Statut de salariés', 'Frontalier', 'Rub 6082', '6082Taux']
target = 'anomalie_csg'

# 3. Convertir les variables catégorielles en numériques (Statut de salariés, Frontalier)
label_encoder_statut = LabelEncoder()
df['Statut de salariés'] = label_encoder_statut.fit_transform(df['Statut de salariés'])

label_encoder_frontalier = LabelEncoder()
df['Frontalier'] = label_encoder_frontalier.fit_transform(df['Frontalier'])

# Sauvegarder les encodeurs
joblib.dump(label_encoder_statut, 'label_encoder_statut.pkl')
joblib.dump(label_encoder_frontalier, 'label_encoder_frontalier.pkl')

# 4. Préparer les données pour le modèle
X = df[features]
y = df[target]

# 5. Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 6. Créer et entraîner le modèle RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 7. Faire des prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)

# 8. Évaluer la performance du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy * 100))

# Rapport de classification pour plus de détails
print("Rapport de classification :\n", classification_report(y_test, y_pred))


Précision du modèle sur les données de test : 100.00%
Rapport de classification :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     87607
           1       1.00      1.00      1.00      9528

    accuracy                           1.00     97135
   macro avg       1.00      1.00      1.00     97135
weighted avg       1.00      1.00      1.00     97135



In [32]:
import joblib

# Exporter le modèle Random Forest entraîné
joblib.dump(rf_classifier, 'random_forest_model_6082_new.pkl')

['random_forest_model_6082_new.pkl']

In [40]:
import pandas as pd
import joblib

# 1. Charger le modèle entraîné
rf_classifier_loaded = joblib.load('random_forest_model_6082_new.pkl')

# 2. Charger la nouvelle base de données (sans la colonne 'anomalie_csg')
new_df = pd.read_csv('base_anomalies_csg.csv')

# 3. Charger les encodeurs
label_encoder_statut = joblib.load('label_encoder_statut.pkl')
label_encoder_frontalier = joblib.load('label_encoder_frontalier.pkl')

# 4. Sélectionner les mêmes colonnes que celles utilisées pour l'entraînement
features_new = new_df[['Statut de salariés', 'Frontalier', 'Rub 6082', '6082Taux']]

# 5. Encodage des variables catégorielles (Statut de salariés, Frontalier)
features_new['Statut de salariés'] = label_encoder_statut.transform(features_new['Statut de salariés'])
features_new['Frontalier'] = label_encoder_frontalier.transform(features_new['Frontalier'])

# 6. Vérifier que le nombre de colonnes correspond bien à celles utilisées pour l'entraînement
# Si certaines colonnes manquent (cas rare), on les ajoute avec des valeurs par défaut
missing_cols = set(X.columns) - set(features_new.columns)  # 'X' ici représente les colonnes d'entraînement
for c in missing_cols:
    features_new[c] = 0  # Ajouter des colonnes manquantes avec une valeur par défaut

# 7. Réarranger les colonnes dans le même ordre que lors de l'entraînement
features_new = features_new[X.columns]

# 8. Utiliser le modèle pour prédire les anomalies sur la nouvelle base
predictions = rf_classifier_loaded.predict(features_new)

# 9. Ajouter la colonne 'anomalie_predite' pour indiquer les prédictions faites par le modèle
new_df['anomalie_predite'] = predictions

# 10. Sauvegarder la nouvelle base de données avec les prédictions
new_df.to_csv('nouvelle_base_avec_predictions.csv', index=False)

print("Prédictions ajoutées à la nouvelle base de données.")



Prédictions ajoutées à la nouvelle base de données.


C:\Users\ouiss\AppData\Local\Temp\ipykernel_7932\1291690022.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_new['Statut de salariés'] = label_encoder_statut.transform(features_new['Statut de salariés'])
C:\Users\ouiss\AppData\Local\Temp\ipykernel_7932\1291690022.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_new['Frontalier'] = label_encoder_frontalier.transform(features_new['Frontalier'])
